#Scrapping SnP Historic Stock Price Data from the Internet using python



In [ ]:
import requests
from PIL import Image
from io import BytesIO


url = "https://www.investopedia.com/thmb/gMnydN0LRCzn6eSAb_nVKuEGtQA=/680x440/filters:no_upscale():max_bytes(150000):strip_icc()/investing-9-5bfc2b2c4cedfd0026c10705.jpg"
response = requests.get(url)
img = Image.open(BytesIO(response.content))
img.show()


In this project, I will be scrapping historical Stock price data from the internet. To do this, I shall use the yfinance library and the data will be scraped from the S&P 500 (Standard & Poor's 500) which is a stock market index that tracks the performance of 500 large publicly traded companies in the United States. It is considered to be one of the most widely-followed and closely watched equity indices in the world. The S&P 500 index is a relatively reliable indicator of the position of the American economy. 

In [6]:
# Install the yfinance library via pip
! pip install yfinance


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 KB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 KB 9.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting uninstall: html5lib
    Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
   

In [7]:
# Import the Libraries neccessary for the project
import pandas as pd
import yfinance as yf

## Get the Initial S&P table table from wikipedia
Wikipedia is a great place to srape data from as the wikipedia project is open-source and most of the restriction that exists in other sites do not exist in Wikipedia. We will obtain the current list of companies in the S&P along with theitr tickers from wikipedia.

In [8]:
# Get the page from wikipedia and assign it to the sp500ulr variable
sp500url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

# read sp500url to pd and name the data frame "data_table"
data_table = pd.read_html(sp500url)

In [ ]:
data_table

[    Symbol              Security SEC filings             GICS Sector  \
 0      MMM                    3M     reports             Industrials   
 1      AOS           A. O. Smith     reports             Industrials   
 2      ABT                Abbott     reports             Health Care   
 3     ABBV                AbbVie     reports             Health Care   
 4      ACN             Accenture     reports  Information Technology   
 ..     ...                   ...         ...                     ...   
 498    YUM           Yum! Brands     reports  Consumer Discretionary   
 499   ZBRA    Zebra Technologies     reports  Information Technology   
 500    ZBH         Zimmer Biomet     reports             Health Care   
 501   ZION  Zions Bancorporation     reports              Financials   
 502    ZTS                Zoetis     reports             Health Care   
 
                       GICS Sub-Industry    Headquarters Location  Date added  \
 0              Industrial Conglomerates 

In [9]:
# Check the number of tables that are returned
len(data_table)

2

Checking the length of tables returned from, We can see that there are two tables returned from that page. Only the first table is relevant to us though. We would filter out the relevant table. 

In [10]:
# Use the index "0" to filter the 1st table.
data_table[0]

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...,...
498,YUM,Yum! Brands,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
499,ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
500,ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
501,ZION,Zions Bancorporation,reports,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


## Extract Tickers
The tickers are the unique identifiers of the stocks. this is the information that we can use to identify the stocks that are part of the S&P index when we want to extract the historical price data from Yahoo Finance. We will extract these tickers by creating a list from the ticker objects.

In [11]:
# Create a list fronm the data table that contains all the ticker symbols in the S&P 500
tickers = data_table[0]["Symbol"].tolist()

In [ ]:
tickers

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ATVI',
 'ADM',
 'ADBE',
 'ADP',
 'AAP',
 'AES',
 'AFL',
 'A',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AMD',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'AZO',
 'AVB',
 'AVY',
 'BKR',
 'BALL',
 'BAC',
 'BBWI',
 'BAX',
 'BDX',
 'WRB',
 'BRK.B',
 'BBY',
 'BIO',
 'TECH',
 'BIIB',
 'BLK',
 'BK',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF.B',
 'CHRW',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'CNC',
 'CNP',
 'CDAY',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CMA

In [ ]:
# Check the len of the ticker list to confirm that it is at least 500 objects long
len(tickers)

503

## Downloading Data using yfinance
To download data from Yahoo finance using the yfinance library, you need to simply call up the library and enter the neccesary variables that would determine the data that you would pull. In this test, we shall be pulling data from Tesla with ticker TSLA,and we want the data returned to start from "2022-6-14" and end at "2022-6-18".

In [ ]:
# pull data from yahoo finance using the yf library
yf.download ("TSLA", start = "2022-6-14", end = "2022-6-18" )

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-06-14 00:00:00-04:00,218.286667,226.330002,211.736664,220.889999,220.889999,97988700
2022-06-15 00:00:00-04:00,220.916672,235.663330,218.149994,233.000000,233.000000,119131800
2022-06-16 00:00:00-04:00,222.736664,225.166672,208.693329,213.100006,213.100006,107390700
2022-06-17 00:00:00-04:00,213.433334,220.970001,213.196671,216.759995,216.759995,92641800


## Deciding and editing  the details of historic data pulled
The data pulled is the detailed stock data for TSLA and it contains the opening price, the highest price, the lowest price the closing price and the Adjusted Closing price as seen in the table. this is too much information for the scope of this project though. We are interested in only the Adjusted closing price as we want the information to be able to tell how stock prices move over a relatively long period. For the general data required, we will only pull the Adjusted closing price ("Adj Close").

In [12]:
# pull the price data for all of the stocks, using the ticker list for 2 days and assign the it to snp_prices
snp_prices = yf.download (tickers, start = "2022-6-14", end = "2022-6-18" )["Adj Close"]

[*********************100%***********************]  503 of 503 completed

3 Failed downloads:
- GEHC: Data doesn't exist for startDate = 1655179200, endDate = 1655524800
- BRK.B: No timezone found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted


In [ ]:
snp_prices

,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACGL,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2022-06-14 00:00:00-04:00,115.432648,12.95,166.725845,132.356277,133.700302,140.340164,101.543419,44.549999,271.819305,370.820007,...,58.009998,65.005516,94.415192,35.173435,76.045670,109.191727,104.599754,300.739990,51.043770,156.808502
2022-06-15 00:00:00-04:00,115.890366,13.31,167.513382,135.018158,134.156921,141.433380,103.378189,43.990002,277.816498,376.920013,...,59.340000,64.975914,93.226402,34.946125,75.995964,110.845993,106.288933,308.359985,52.324299,157.076828
2022-06-16 00:00:00-04:00,114.387878,12.16,162.275833,129.664490,135.186722,140.091705,100.685226,42.230000,267.649933,365.079987,...,54.490002,64.087860,89.787758,34.333385,72.636040,108.111984,102.582672,292.119995,50.915718,156.401093
2022-06-17 00:00:00-04:00,112.149063,12.94,164.590561,131.159927,134.341507,139.813416,101.138977,43.400002,272.076202,360.790009,...,54.959999,63.061665,84.610161,34.916477,72.476990,108.468590,101.996437,288.459991,51.477180,157.832047


## Failed Downloads
From the download summary, we can tell that there are 3 failed downloads listed with the reason for the failure stated too as:
- GEHC: Data doesn't exist for startDate = 1655179200, endDate = 1655524800
- BRK.B: No timezone found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted. 

In summary, GEHC is a new entrant to the S&P 500 and therefore the data that exists for it is insufficient for the time range. We shall investigate the reason for the failure on the other two data sets.

In [13]:
# Use the isna method to confirm that the failed stocks are indeed without data
snp_prices.isna().any()

A       False
AAL     False
AAP     False
AAPL    False
ABBV    False
        ...  
YUM     False
ZBH     False
ZBRA    False
ZION    False
ZTS     False
Length: 503, dtype: bool

In [14]:
# Filter for the stocks that have null values
snp_prices.loc[:, snp_prices.isna().any()]

,BF.B,BRK.B,GEHC
Date,,,
2022-06-14 00:00:00-04:00,NaN,NaN,NaN
2022-06-15 00:00:00-04:00,NaN,NaN,NaN
2022-06-16 00:00:00-04:00,NaN,NaN,NaN
2022-06-17 00:00:00-04:00,NaN,NaN,NaN


We can see that indeed the BF.B, BRK.B and GEHC stocks contain null values as the data in them.

## Handling Failed Downloads
The attempt to download information may fail for a variety of reasons. In our example, the first stock represented by the ticker GEHC failed because there is no information for the time in consideration, while the second and third represented by the tickers BRK.B, and BF.b failed to download because due to discrepancies in the ticker symbols. Our investigations# revealed that the stocks had a different symbol as the ticker downloaded from wikepedia from the ticker symbol recorded against the same stock at Yahoo finance.Instead of a hyphen, the wikepedia ticker used a full stop within the ticker symbol. This discrepancy made it impossible for the ticker on the list to be mapped to its corresponding stock. To solve this, we must update the ticker list with the information at Yahoo finance.

In [15]:
 # Test to confirm that  the ticker from yahoo finance works as it should
 yf.download("BF-B")["Adj Close"]

[*********************100%***********************]  1 of 1 completed


Date
1980-03-17 00:00:00-05:00     0.198797
1980-03-18 00:00:00-05:00     0.198797
1980-03-19 00:00:00-05:00     0.201457
1980-03-20 00:00:00-05:00     0.202121
1980-03-21 00:00:00-05:00     0.199462
                               ...    
2023-01-24 00:00:00-05:00    67.160004
2023-01-25 00:00:00-05:00    68.349998
2023-01-26 00:00:00-05:00    67.160004
2023-01-27 00:00:00-05:00    66.839996
2023-01-30 00:00:00-05:00    66.000000
Name: Adj Close, Length: 10810, dtype: float64

In [16]:
 yf.download("BRK-B")["Adj Close"]

[*********************100%***********************]  1 of 1 completed


Date
1996-05-09 00:00:00-04:00     23.200001
1996-05-10 00:00:00-04:00     24.000000
1996-05-13 00:00:00-04:00     23.900000
1996-05-14 00:00:00-04:00     23.600000
1996-05-15 00:00:00-04:00     23.200001
                                ...    
2023-01-24 00:00:00-05:00    311.299988
2023-01-25 00:00:00-05:00    311.899994
2023-01-26 00:00:00-05:00    310.950012
2023-01-27 00:00:00-05:00    309.170013
2023-01-30 00:00:00-05:00    307.329987
Name: Adj Close, Length: 6727, dtype: float64

In [17]:
 yf.download("GEHC")["Adj Close"]

[*********************100%***********************]  1 of 1 completed


Date
2022-12-15 00:00:00-05:00    60.000000
2022-12-16 00:00:00-05:00    56.740002
2022-12-19 00:00:00-05:00    56.500000
2022-12-20 00:00:00-05:00    57.209999
2022-12-21 00:00:00-05:00    56.959999
2022-12-22 00:00:00-05:00    57.599998
2022-12-23 00:00:00-05:00    57.500000
2022-12-27 00:00:00-05:00    57.450001
2022-12-28 00:00:00-05:00    57.500000
2022-12-29 00:00:00-05:00    58.000000
2022-12-30 00:00:00-05:00    58.380001
2023-01-03 00:00:00-05:00    56.000000
2023-01-04 00:00:00-05:00    60.490002
2023-01-05 00:00:00-05:00    59.000000
2023-01-06 00:00:00-05:00    58.950001
2023-01-09 00:00:00-05:00    58.099998
2023-01-10 00:00:00-05:00    60.000000
2023-01-11 00:00:00-05:00    64.989998
2023-01-12 00:00:00-05:00    65.650002
2023-01-13 00:00:00-05:00    65.440002
2023-01-17 00:00:00-05:00    63.330002
2023-01-18 00:00:00-05:00    62.080002
2023-01-19 00:00:00-05:00    63.939999
2023-01-20 00:00:00-05:00    66.769997
2023-01-23 00:00:00-05:00    69.519997
2023-01-24 00:00:00-

Note that the start date for the price information on GEHC started on 2022-12-15, while the date range for our request is much earlier than that. Since the data simply does not exist, we would have no other choice but to drop that particular stock. We would however replace the wrong tickers on the tickers list with the correct tickers so that the data will be pulled.

In [18]:
#This for loop iterates through the tickers list and updates the tickers with the correct symbol
for i in range(len(tickers)):
    if tickers[i] == 'BRK.B':
        tickers[i] = 'BRK-B'
    elif tickers[i] == 'BF.B':
        tickers[i] = 'BF-B'    

In [19]:
#Confirm to see that the correct tickers are in the tickers list
'BF-B' in tickers

True

In [20]:
'BRK-B' in tickers

True

In [21]:
#Confirm to see that the wrong tickers are off the tickers list
'BF.B' in tickers

False

In [22]:
'BRK.B' in tickers

False

In [23]:
# remove the GEHC stock from the list
tickers.remove("GEHC")

##Download the Final Stock Data
Your stock data is now ready to download. All you need to do now is to specify the date range for which data is required and download the data accordingly. In our case, we are interested in the data for a 6year period, starting at 1st january, 2017 and ending at 1st January, 2023.



In [24]:
# Specify the date range and download the required data
snp_prices_Updated = yf.download (tickers, start = "2017-1-1", end = "2023-1-1" )["Adj Close"]

[*********************100%***********************]  502 of 502 completed


In [25]:
# View the DF of the downloadedd data
snp_prices_Updated

,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACGL,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2017-01-03 00:00:00-05:00,44.367443,44.741226,160.039307,27.174759,47.287483,74.811691,34.993328,28.629999,105.776184,103.480003,...,81.032700,34.193760,67.322067,55.654686,46.163960,56.863945,95.754021,86.250000,37.288666,51.470894
2017-01-04 00:00:00-05:00,44.949589,45.127758,161.352661,27.144341,47.954254,76.668190,35.271118,28.833332,106.030502,104.139999,...,83.645454,34.345295,66.581390,56.110409,46.851982,57.070866,96.634354,87.029999,37.824085,51.970329
2017-01-05 00:00:00-05:00,44.415157,44.345032,161.240097,27.282383,48.317955,75.780685,35.575790,28.540001,104.441040,105.910004,...,84.720222,34.345295,65.588860,55.379356,46.424282,57.259789,97.255226,84.750000,37.210957,51.797455
2017-01-06 00:00:00-05:00,45.798954,44.654255,159.129364,27.586535,48.333111,76.794968,36.543602,28.823334,105.630852,108.300003,...,85.637459,34.446293,65.551819,55.341381,46.098862,57.952461,97.264488,85.959999,37.452755,51.960728
2017-01-09 00:00:00-05:00,45.942112,45.494972,159.035522,27.839207,48.651333,77.410789,36.507771,28.406668,104.450111,108.570000,...,85.933945,33.924381,64.470398,55.512276,45.903614,58.114395,99.154907,85.970001,37.046879,51.816658
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23 00:00:00-05:00,149.007767,12.710000,143.279999,131.860001,161.564163,170.009995,107.692398,63.380001,265.006012,338.450012,...,80.720001,70.446365,108.680000,31.701571,109.730003,128.899994,126.690002,248.220001,48.450001,145.418304
2022-12-27 00:00:00-05:00,149.327301,12.530000,145.020004,130.029999,161.455200,169.000000,108.080643,63.619999,264.229187,335.089996,...,84.330002,71.082001,110.190002,31.940601,110.720001,129.899994,127.279999,251.000000,48.840000,144.959396
2022-12-28 00:00:00-05:00,147.869461,12.320000,145.300003,126.040001,160.702347,167.360001,107.343979,62.599998,262.048126,328.329987,...,80.089996,70.570000,108.379997,30.855000,108.940002,129.309998,125.989998,246.839996,47.970001,143.492844


## Exporting your data to CSV
We now have our data that has all the adjusted closing prices of all the stocks in the S&P 500 from the 1st of January 2017 to the 1st of Jhanuary 2023. The final task is to export it to a handy .csv file that can be used for any analysis. This can be done using the `to_csv` method.

In [26]:
snp_prices_Updated.to_csv('S&P_500_Historical_Price.csv', index=True)
